<a href="https://colab.research.google.com/github/ParalelaUnsaac/2020-2/blob/main/examen_sustitutorio_2020_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [5]:
!pip install ipython-autotime

%load_ext autotime

time: 176 µs (started: 2021-03-15 18:02:16 +00:00)


**Pregunta 1 y 2**

In [6]:
#pagina 175 del libro https://apps2.mdp.ac.id/perpustakaan/ebook/Karya%20Umum/Portable_Shared_Memory_Parallel_Programming.pdf
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char* argv[])
{
  int n=6;
  int sum=0;
  int a[n];
  int TID, sumLocal,i;
  for(int j = 0; j < n; j++)
   a[j]=j;
 
  #pragma omp parallel shared(n,a,sum) private(TID,sumLocal)
  {
    TID = omp_get_thread_num();
    sumLocal = 0;
    #pragma omp for
      for (i=0; i<n; i++)
      sumLocal += a[i];
      printf("Suma parcial fuera del Critical: TID=%d: sumLocal=%d \\n", TID, sumLocal);
    #pragma omp critical
     {
        sum += sumLocal;
        printf("TID=%d: sumLocal=%d sum = %d\\n",TID,sumLocal,sum);
     }
  } /*-- End of parallel region --*/
  
  printf("Value of sum after parallel region: %d\\n",sum);
}"""

#creando un archivo para guardar el código
text_file = open("codeA.c", "w")

#Guardar el code en el text_file
text_file.write(code)

#cerrar el archivo text_file
text_file.close()

#definir cuantos hilos se van a usar
%env OMP_NUM_THREADS=3

#ejecutar nuestro código paralelo llamado codeA
!gcc -o Ejecutar -fopenmp codeA.c
!./Ejecutar

env: OMP_NUM_THREADS=3
Suma parcial fuera del Critical: TID=1: sumLocal=5 
TID=1: sumLocal=5 sum = 5
Suma parcial fuera del Critical: TID=0: sumLocal=1 
Suma parcial fuera del Critical: TID=2: sumLocal=9 
TID=0: sumLocal=1 sum = 6
TID=2: sumLocal=9 sum = 15
Value of sum after parallel region: 15
time: 230 ms (started: 2021-03-15 18:02:19 +00:00)


**Pregunta 3**

In [17]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>

__global__ void addKernel(int* c, const int* a, const int* b, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        c[i] = a[i] + b[i];
        
    }
}

// Helper function for using CUDA to add vectors in parallel.
void addWithCuda(int* c, const int* a, const int* b, int size) {
    int* dev_a = nullptr;
    int* dev_b = nullptr;
    int* dev_c = nullptr;

    // Allocate GPU buffers for three vectors (two input, one output)
    cudaMalloc((void**)&dev_c, size * sizeof(int));
    cudaMalloc((void**)&dev_a, size * sizeof(int));
    cudaMalloc((void**)&dev_b, size * sizeof(int));

    // Copy input vectors from host memory to GPU buffers.
    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);

    // Launch a kernel on the GPU with one thread for each element.
    // 2 is number of computational blocks and (size + 1) / 2 is a number of threads in a block
    addKernel<<<2, (size + 1) / 2>>>(dev_c, dev_a, dev_b, size);
    
    // cudaDeviceSynchronize waits for the kernel to finish, and returns
    // any errors encountered during the launch.
    cudaDeviceSynchronize();

    // Copy output vector from GPU buffer to host memory.
    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);
}

int main(int argc, char** argv) {
    
    const int arraySize = 1000;
    int a[arraySize]={0};
    int b[arraySize]={0};
    for(int j = 0; j < arraySize; j++){
      a[j] = j;
      b[j] = j;
    }
    
    int c[arraySize] = { 0 };

    addWithCuda(c, a, b, arraySize);

    for(int j = 0; j < arraySize; j++){
      printf("a = %d ", a[j]);
    }
    printf("\n");
    for(int j = 0; j < arraySize; j++){
      printf("b = %d ", b[j]);
    }
    printf("\n");
    for(int j = 0; j < arraySize; j++){
      printf("c = %d ", c[j]);
    }

    cudaDeviceReset();

    return 0;
}

a = 0 a = 1 a = 2 a = 3 a = 4 a = 5 a = 6 a = 7 a = 8 a = 9 a = 10 a = 11 a = 12 a = 13 a = 14 a = 15 a = 16 a = 17 a = 18 a = 19 a = 20 a = 21 a = 22 a = 23 a = 24 a = 25 a = 26 a = 27 a = 28 a = 29 a = 30 a = 31 a = 32 a = 33 a = 34 a = 35 a = 36 a = 37 a = 38 a = 39 a = 40 a = 41 a = 42 a = 43 a = 44 a = 45 a = 46 a = 47 a = 48 a = 49 a = 50 a = 51 a = 52 a = 53 a = 54 a = 55 a = 56 a = 57 a = 58 a = 59 a = 60 a = 61 a = 62 a = 63 a = 64 a = 65 a = 66 a = 67 a = 68 a = 69 a = 70 a = 71 a = 72 a = 73 a = 74 a = 75 a = 76 a = 77 a = 78 a = 79 a = 80 a = 81 a = 82 a = 83 a = 84 a = 85 a = 86 a = 87 a = 88 a = 89 a = 90 a = 91 a = 92 a = 93 a = 94 a = 95 a = 96 a = 97 a = 98 a = 99 a = 100 a = 101 a = 102 a = 103 a = 104 a = 105 a = 106 a = 107 a = 108 a = 109 a = 110 a = 111 a = 112 a = 113 a = 114 a = 115 a = 116 a = 117 a = 118 a = 119 a = 120 a = 121 a = 122 a = 123 a = 124 a = 125 a = 126 a = 127 a = 128 a = 129 a = 130 a = 131 a = 132 a = 133 a = 134 a = 135 a = 136 a = 137 a = 13

In [31]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char* argv[])
{
  int n=1000;
  int sum=0;
  int a[n],b[n],c[n];
  int TID, sumLocal,i;
  for(int j = 0; j < n; j++){
   a[j]=j;
   b[j]=j;
 }
  #pragma omp parallel shared(n,a,sum) private(TID,sumLocal)
  {
    TID = omp_get_thread_num();
    sumLocal = 0;
    #pragma omp for
      for (i=0; i<n; i++)
      c[i]= a[i]+b[i];
  
  } /*-- End of parallel region --*/
  
  for(int j = 0; j < n; j++){
      printf("a = %d ", a[j]);
  }
  printf("\\n");
  for(int j = 0; j < n; j++){
      printf("b = %d ",b[j]);
  }
  printf("\\n");
  for(int j = 0; j < n; j++){
      printf("c = %d ", c[j]);
  }
  printf("\\n");
 
}"""

#creando un archivo para guardar el código
text_file = open("codeA.c", "w")

#Guardar el code en el text_file
text_file.write(code)

#cerrar el archivo text_file
text_file.close()

#definir cuantos hilos se van a usar
%env OMP_NUM_THREADS=3

#ejecutar nuestro código paralelo llamado codeA
!gcc -o Ejecutar -fopenmp codeA.c
!./Ejecutar

env: OMP_NUM_THREADS=3
a = 0 a = 1 a = 2 a = 3 a = 4 a = 5 a = 6 a = 7 a = 8 a = 9 a = 10 a = 11 a = 12 a = 13 a = 14 a = 15 a = 16 a = 17 a = 18 a = 19 a = 20 a = 21 a = 22 a = 23 a = 24 a = 25 a = 26 a = 27 a = 28 a = 29 a = 30 a = 31 a = 32 a = 33 a = 34 a = 35 a = 36 a = 37 a = 38 a = 39 a = 40 a = 41 a = 42 a = 43 a = 44 a = 45 a = 46 a = 47 a = 48 a = 49 a = 50 a = 51 a = 52 a = 53 a = 54 a = 55 a = 56 a = 57 a = 58 a = 59 a = 60 a = 61 a = 62 a = 63 a = 64 a = 65 a = 66 a = 67 a = 68 a = 69 a = 70 a = 71 a = 72 a = 73 a = 74 a = 75 a = 76 a = 77 a = 78 a = 79 a = 80 a = 81 a = 82 a = 83 a = 84 a = 85 a = 86 a = 87 a = 88 a = 89 a = 90 a = 91 a = 92 a = 93 a = 94 a = 95 a = 96 a = 97 a = 98 a = 99 a = 100 a = 101 a = 102 a = 103 a = 104 a = 105 a = 106 a = 107 a = 108 a = 109 a = 110 a = 111 a = 112 a = 113 a = 114 a = 115 a = 116 a = 117 a = 118 a = 119 a = 120 a = 121 a = 122 a = 123 a = 124 a = 125 a = 126 a = 127 a = 128 a = 129 a = 130 a = 131 a = 132 a = 133 a = 134 a = 135